BÀI 1 :

In [1]:
import sqlite3

conn = sqlite3.connect('data1.db') 
cursor = conn.cursor()

# Tạo bảng với các cột A và B
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Bang1 (
        A REAL,
        B REAL
    );
""")

# Thêm dữ liệu vào bảng
cursor.executemany("""
    INSERT INTO Bang1 (A, B)
    VALUES (?, ?);
""", [
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 6)
])

conn.commit()

conn.close()

# Mở lại kết nối 
conn = sqlite3.connect('data1.db')
cursor = conn.cursor()

# Thực hiện truy vấn SQL để tính hệ số tương quan Pearson
cursor.execute("""
    SELECT 
        (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) / 
        (
            (COUNT(*) * SUM(A * A) - POWER(SUM(A), 2)) * 
            (COUNT(*) * SUM(B * B) - POWER(SUM(B), 2))
        ) AS correlation_coefficient
    FROM 
        Bang1;
""")

# Lấy kết quả
correlation_coefficient = cursor.fetchone()[0]

conn.close()

print(f"Hệ số tương quan Pearson giữa A và B là: {correlation_coefficient}")


Hệ số tương quan Pearson giữa A và B là: 0.005


BÀI 2 :

In [2]:
import pandas as pd
import scipy.stats as stats

# Dữ liệu mẫu cho các ngày và các mẫu xe
data = {'Day 1': [8, 9, 7], 'Day 2': [7.5, 8.5, 7], 'Day 3': [6, 7, 8], 'Day 4': [7, 6, 5]}
df = pd.DataFrame(data, index=['A', 'B', 'C'])

# Chuyển dữ liệu sang dạng quan hệ 
relationship_data = pd.melt(df.reset_index(), id_vars=['index'], value_vars=df.columns)
relationship_data.columns = ['Mẫu', 'Ngày', 'Điểm']

print("Dữ liệu dạng quan hệ:")
print(relationship_data)

# Kiểm định ANOVA để kiểm tra sự khác biệt giữa các mẫu A, B, C
A = [8, 7.5, 6, 7]
B = [9, 8.5, 7, 6]
C = [7, 7, 8, 5]
f_statistic, p_value = stats.f_oneway(A, B, C)

# Kết quả kiểm định ANOVA
if p_value < 0.05:
    print("\nCó sự khác biệt giữa các mẫu A, B và C.")
else:
    print("\nKhông có sự khác biệt giữa các mẫu A, B và C.")

# Tạo bảng tần suất cho kiểm định chi-square
contingency_table = pd.crosstab(relationship_data['Mẫu'], relationship_data['Ngày'])

# Kiểm định Chi-square
chi2_stat, p_val, dof, expected = stats.chi2_contingency(contingency_table)

# Kết quả kiểm định Chi-square
if p_val < 0.05:
    print("\nCó sự phụ thuộc giữa ngày và mẫu xe.")
else:
    print("\nKhông có sự phụ thuộc giữa ngày và mẫu xe.")


Dữ liệu dạng quan hệ:
   Mẫu   Ngày  Điểm
0    A  Day 1   8.0
1    B  Day 1   9.0
2    C  Day 1   7.0
3    A  Day 2   7.5
4    B  Day 2   8.5
5    C  Day 2   7.0
6    A  Day 3   6.0
7    B  Day 3   7.0
8    C  Day 3   8.0
9    A  Day 4   7.0
10   B  Day 4   6.0
11   C  Day 4   5.0

Không có sự khác biệt giữa các mẫu A, B và C.

Không có sự phụ thuộc giữa ngày và mẫu xe.


BÀI 3:

In [3]:
import pandas as pd

# Giả sử có một DataFrame với cột departure_time
df = pd.DataFrame({'departure_time': [830, 1430, 1111, 630]})

# Chuyển đổi giá trị thời gian từ số nguyên sang định dạng HH:MM
df['formatted_time'] = df['departure_time'].apply(lambda x: f"{x // 100:02d}:{x % 100:02d}")

print(df)

   departure_time formatted_time
0             830          08:30
1            1430          14:30
2            1111          11:11
3             630          06:30


BÀI 4 :

In [4]:
import sqlite3
import numpy as np

conn = sqlite3.connect('data4.db') 
cursor = conn.cursor()

# Giả sử có một bảng với cột value chứa các giá trị
cursor.execute("""
    CREATE TABLE IF NOT EXISTS data_table (
        id INTEGER PRIMARY KEY,
        value INTEGER
    )
""")

# Thêm dữ liệu mẫu vào bảng
cursor.executemany("""
    INSERT INTO data_table (value)
    VALUES (?)
""", [(830,), (1111,), (930,), (1230,), (1100,)])

conn.commit()

# Lấy tất cả giá trị từ bảng và tính trung vị 
cursor.execute("""
    SELECT value
    FROM data_table
""")
values = [row[0] for row in cursor.fetchall()]

# Tính trung vị
median_value = np.median(values)

# Tính độ lệch tuyệt đối so với trung vị
deviations = [abs(value - median_value) for value in values]

# Tính MAD (trung vị của các độ lệch tuyệt đối)
mad_value = np.median(deviations)

# Lọc các giá trị ngoại lệ (giá trị có độ lệch tuyệt đối > 1.5 * MAD)
outliers = []
for i, deviation in enumerate(deviations):
    if deviation > 1.5 * mad_value:
        outliers.append((values[i], deviation))

print(f"Trung vị (Median): {median_value}")
print(f"MAD: {mad_value}")
print("Các ngoại lệ:")
for outlier in outliers:
    print(f"Value: {outlier[0]}, Absolute Deviation: {outlier[1]}")

conn.close()

Trung vị (Median): 1100.0
MAD: 134.0
Các ngoại lệ:
Value: 111, Absolute Deviation: 989.0
Value: 1444, Absolute Deviation: 344.0
Value: 830, Absolute Deviation: 270.0
Value: 1445, Absolute Deviation: 345.0
Value: 830, Absolute Deviation: 270.0


BÀI 5 :

In [10]:
import sqlite3

conn = sqlite3.connect('data5.db') 
cursor = conn.cursor()

# Giả sử có một bảng Patient với cột last_name, weight, và height
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Patient (
        id INTEGER PRIMARY KEY,
        last_name TEXT,
        weight REAL,
        height REAL
    )
""")

cursor.executemany("""
    INSERT INTO Patient (last_name, weight, height)
    VALUES (?, ?, ?)
""", [
    ('Smith', 70, 170),
    ('Johnson', 70, 175),
    ('Smith', 70, 165),
    ('Johnson', 80, 180),
    ('Smith', 72, 168)
])

conn.commit()

# Truy vấn để tính khoảng cách kết hợp Boolean giữa last_name và weight
cursor.execute("""
    SELECT 
        p1.last_name AS last_name1,
        p1.weight AS weight1,
        p2.last_name AS last_name2,
        p2.weight AS weight2,
        CASE 
            WHEN p1.last_name = p2.last_name THEN 0 ELSE 1 
        END AS last_name_distance,
        CASE 
            WHEN p1.weight = p2.weight THEN 0 ELSE 1 
        END AS weight_distance,
        (CASE 
            WHEN p1.last_name = p2.last_name THEN 0 ELSE 1 
        END + CASE 
            WHEN p1.weight = p2.weight THEN 0 ELSE 1 
        END) AS total_distance
    FROM Patient p1
    CROSS JOIN Patient p2
    WHERE p1.id < p2.id;
""")

# Lấy kết quả và kiểm tra tổng khoảng cách
matches = cursor.fetchall()

print("So sánh các cặp người:")
for match in matches:
    last_name_distance = match[4]
    weight_distance = match[5]
    total_distance = match[6]
    
    print(f"{match[0]} và {match[2]} (Trọng lượng: {match[1]}, {match[3]}) => Total Distance: {total_distance}")
    if total_distance == 0:
        print(f" -> Cả hai là một người.")
    else:
        print(f" -> Cả hai không phải là một người.")

conn.close()


So sánh các cặp người:
Smith và Johnson (Trọng lượng: 70.0, 70.0) => Total Distance: 1
 -> Cả hai không phải là một người.
Smith và Smith (Trọng lượng: 70.0, 70.0) => Total Distance: 0
 -> Cả hai là một người.
Smith và Johnson (Trọng lượng: 70.0, 80.0) => Total Distance: 2
 -> Cả hai không phải là một người.
Smith và Smith (Trọng lượng: 70.0, 72.0) => Total Distance: 1
 -> Cả hai không phải là một người.
Smith và Smith (Trọng lượng: 70.0, 70.0) => Total Distance: 0
 -> Cả hai là một người.
Smith và Johnson (Trọng lượng: 70.0, 70.0) => Total Distance: 1
 -> Cả hai không phải là một người.
Smith và Smith (Trọng lượng: 70.0, 70.0) => Total Distance: 0
 -> Cả hai là một người.
Smith và Johnson (Trọng lượng: 70.0, 80.0) => Total Distance: 2
 -> Cả hai không phải là một người.
Smith và Smith (Trọng lượng: 70.0, 72.0) => Total Distance: 1
 -> Cả hai không phải là một người.
Johnson và Smith (Trọng lượng: 70.0, 70.0) => Total Distance: 1
 -> Cả hai không phải là một người.
Johnson và Johnson (